# Generate dataset

The aim of this notebook is to create a varied dataset with atomRDF that can be used testing quering methods, and also to show to variety of tasks that can be done. This should be runnable top to bottom.

In [4]:
%config IPCompleter.evaluation='unsafe'

Create a project

In [5]:
from pyiron_atomistics import Project
from atomrdf import KnowledgeGraph, System
import numpy as np

In [6]:
project = 'data_gen_1'
pr = Project(project)

In [7]:
kg = KnowledgeGraph(store='db', store_file=f'{project}.db')
kg.enable_workflow(pr, workflow_environment='pyiron')

Regular structures

In [8]:
struct_Fe = System.create.element.Fe(graph=kg)
struct_Si = System.create.element.Si(graph=kg)
struct_l12 = System.create.lattice.l12(element=['Al', 'Ni'], 
                         lattice_constant=3.57, graph=kg)


In [9]:
struct_gb_1 = System.create.defect.grain_boundary(axis=[0,0,1], 
                        sigma=5, 
                        gb_plane=[3, -1, 0],
                        element='Fe',
                        graph=kg)
struct_gb_2 = System.create.defect.grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-3, 2, 0],
                        element='Fe',
                        graph=kg)

struct_gb_3 = System.create.defect.grain_boundary(axis=[1,1,1], 
                        sigma=19, 
                        gb_plane=[-1, -1, -1],
                        element='Fe',
                        graph=kg)


In [10]:
struct_Fe = System.create.element.Fe(graph=kg)
struct_with_vacancy = struct_Fe.delete(indices=[0], copy_structure=True)
struct_Al = System.create.element.Al(graph=kg)
struct_Al.substitute_atoms('Mg', indices=[0], copy_structure=True)

In [11]:
slip_direction = np.array([1, 0, -1])
slip_plane = np.array([1, 1, 1])
slip_system = [slip_direction, slip_plane]
burgers_vector = 0.5
dislocation_line = np.array([1, 0, -1])
elastic_constant_dict = {'C11': 169, 'C12': 122, 'C44': 75.4}
sys = System.create.defect.dislocation(slip_system,
                                        dislocation_line,
                                        elastic_constant_dict,
                                        burgers_vector=burgers_vector,
                                        element='Cu',
                                        dislocation_type='monopole',
                                        graph=kg,
                                        )

In [12]:
structure = pr.create.structure.annotated_structure.bulk('Cu', cubic=True)
job = pr.create.job.Lammps('j1', delete_existing_job=True, delete_aborted_job=True)
job.structure = structure
job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
job.calc_md(pressure=0, temperature=500)
job.run()
kg.add_workflow(job, workflow_environment='pyiron')

The job j1 was saved and received the ID: 1106


In [13]:
structure = pr.create.structure.annotated_structure.bulk('Cu', cubic=True)
ref_job = pr.create.job.Lammps('j1', delete_existing_job=True, delete_aborted_job=True)
ref_job.structure = structure
ref_job.potential = '2001--Mishin-Y--Cu-1--LAMMPS--ipr1'
ref_job.calc_minimize()
murn_job = ref_job.create_job(pr.job_type.Murnaghan, 'murn_job')
murn_job.input["num_points"] = 5
murn_job.run()
kg.add_workflow(murn_job, workflow_environment='pyiron', add_intermediate_jobs=True)

The job murn_job was saved and received the ID: 1106
The job murn_job_0_9 was saved and received the ID: 1107
The job murn_job_0_95 was saved and received the ID: 1108
The job murn_job_1_0 was saved and received the ID: 1109
The job murn_job_1_05 was saved and received the ID: 1110
The job murn_job_1_1 was saved and received the ID: 1111


In [14]:
struct = pr.create.structure.annotated_structure.bulk('Cu')
refjob = pr.create.job.Lammps('reflmp')
refjob.structure = struct
refjob.potential = '2009--Mendelev-M-I--Cu-Zr--LAMMPS--ipr1'
phono = pr.create.job.PhonopyJob('phono')
phono.ref_job = refjob
quasi = pr.create.job.QuasiHarmonicJob('quasi')
quasi.ref_job = phono
quasi.input["temperature_end"] = 500
quasi.input["temperature_steps"] = 2
quasi.input["axes"]=["x","y","z"]
quasi.input["strains"] = None
quasi.run()
kg.add_workflow(quasi, workflow_environment='pyiron')

The job quasi was saved and received the ID: 1112
The job quasi_0_9 was saved and received the ID: 1113
The job reflmp_0 was saved and received the ID: 1114
The job quasi_0_92 was saved and received the ID: 1115
The job reflmp_0 was saved and received the ID: 1116
The job quasi_0_94 was saved and received the ID: 1117
The job reflmp_0 was saved and received the ID: 1118
The job quasi_0_96 was saved and received the ID: 1119
The job reflmp_0 was saved and received the ID: 1120
The job quasi_0_98 was saved and received the ID: 1121
The job reflmp_0 was saved and received the ID: 1122
The job quasi_1_0 was saved and received the ID: 1123
The job reflmp_0 was saved and received the ID: 1124
The job quasi_1_02 was saved and received the ID: 1125
The job reflmp_0 was saved and received the ID: 1126
The job quasi_1_04 was saved and received the ID: 1127
The job reflmp_0 was saved and received the ID: 1128
The job quasi_1_06 was saved and received the ID: 1129
The job reflmp_0 was saved and re

/home/srmnitc/miniconda3/envs/workflow-rdf/lib/python3.11/site-packages/atomrdf/graph.py:376: UserWarning: asmo:hasValue has a range with unspecified datatype!
  warnings.warn(f"{triple[1].name} has a range with unspecified datatype!")


In [15]:
kg.archive('dataset')